- https://www.kaggle.com/raddar/paris-madness/notebook

In [ ]:
import numpy as np
import pandas as pd
import os
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
from scipy.interpolate import UnivariateSpline
import statsmodels.api as sm
import matplotlib.pyplot as plt
import collections

- boxscore 전체 추가 후 성능측정해보기: 전체를 추가하니 성능이 떨어짐; 적당한 속성을 선택할 필요가 있음
- season_statistics 변수명 변경
- tourney_data에 스코어썸도 머지하기
- feature importance 그려보기


# Load the data

In [ ]:
'''COMPETITION_YEAR = 2019

if COMPETITION_YEAR == 2019:
    tourney_detail = pd.read_csv('input/MDataFiles_Stage1/NCAATourneyDetailedResults.csv')
    tourney_detail_reload = pd.read_csv('input/MDataFiles_Stage1/NCAATourneyDetailedResults.csv')
    seeds = pd.read_csv('input/MDataFiles_Stage1/NCAATourneySeeds.csv')
    regular_detail = pd.read_csv('input/MDataFiles_Stage1/Prelim2019_RegularSeasonDetailedResults.csv')
    regular_detail_reload = pd.read_csv('input/MDataFiles_Stage1/Prelim2019_RegularSeasonDetailedResults.csv')
    submission = pd.read_csv('input/SampleSubmissionStage2.csv')
    
elif COMPETITION_YEAR == 2020:
    tourney_detail = pd.read_csv('input/MDataFiles_Stage1/MNCAATourneyDetailedResults.csv')
    tourney_detail_reload = pd.read_csv('input/MDataFiles_Stage1/MNCAATourneyDetailedResults.csv')
    seeds = pd.read_csv('input/MDataFiles_Stage1/MNCAATourneySeeds.csv')
    regular_detail = pd.read_csv('input/MDataFiles_Stage1/MRegularSeasonDetailedResults.csv')
    regular_detail_reload = pd.read_csv('input/MDataFiles_Stage1/MRegularSeasonDetailedResults.csv')
    submission = pd.read_csv('input/MSampleSubmissionStage1_2020.csv')'''

In [ ]:
# For Kaggle GPU Test

COMPETITION_YEAR = 2019

if COMPETITION_YEAR == 2019:
    tourney_detail = pd.read_csv('/kaggle/input/mens-machine-learning-competition-2019/Stage2DataFiles/NCAATourneyDetailedResults.csv')
    tourney_detail_reload = pd.read_csv('/kaggle/input/mens-machine-learning-competition-2019/Stage2DataFiles/NCAATourneyDetailedResults.csv')
    seeds = pd.read_csv('/kaggle/input/mens-machine-learning-competition-2019/Stage2DataFiles/NCAATourneySeeds.csv')
    regular_detail = pd.read_csv('/kaggle/input/mens-machine-learning-competition-2019/Prelim2019_RegularSeasonDetailedResults.csv')
    regular_detail_reload = pd.read_csv('/kaggle/input/mens-machine-learning-competition-2019/Prelim2019_RegularSeasonDetailedResults.csv')
    submission = pd.read_csv('/kaggle/input/mens-machine-learning-competition-2019/SampleSubmissionStage2.csv')


In [ ]:
tourney_detail.shape

In [ ]:
seeds.shape

In [ ]:
regular_detail.shape

In [ ]:
tourney_detail

In [ ]:
seeds

In [ ]:
regular_detail

In [ ]:
regular_detail.groupby('Season').mean()

In [ ]:
tourney_detail.groupby('Season').mean()

In [ ]:
# Check if all columns are the same between regular_detail and tourney_detail
all(regular_detail.columns == tourney_detail.columns)

# Data Preparation

In [ ]:
regular_detail.columns

In [ ]:
# Swap the features of winning team and losing team
regular_detail_swap = regular_detail[[
    'Season', 'DayNum', 'LTeamID', 'LScore', 'WTeamID', 'WScore', 'WLoc', 'NumOT', 
    'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 
    'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']]

In [ ]:
regular_detail.head()

In [ ]:
regular_detail_swap.head()

In [ ]:
# Change winning location: "Home" to "Away", "Away" to "Home"
regular_detail_swap.loc[regular_detail['WLoc'] == 'H', 'WLoc'] = 'A'
regular_detail_swap.loc[regular_detail['WLoc'] == 'A', 'WLoc'] = 'H'

# Change "WLoc" to "location"
regular_detail.columns.values[6] = 'location'
regular_detail_swap.columns.values[6] = 'location'

In [ ]:
regular_detail.head()

In [ ]:
regular_detail_swap.head()

In [ ]:
regular_detail.columns = [column.replace('W','T1_').replace('L','T2_') for column in regular_detail.columns]
regular_detail_swap.columns = [column.replace('L','T1_').replace('W','T2_') for column in regular_detail.columns]

In [ ]:
regular_detail.head()

In [ ]:
regular_detail_swap.head()

In [ ]:
# Concat regular_detail and regular_detail_swap together
regular_data = pd.concat([regular_detail, regular_detail_swap]).sort_index().reset_index(drop = True)

In [ ]:
regular_data

### Apply function to prepare data

In [ ]:
def prepare_data(df):
    # Swap winnig team's columns and losing team's columns
    df_swap = df[['Season', 'DayNum', 'LTeamID', 'LScore', 'WTeamID', 'WScore', 'WLoc', 'NumOT', 
    'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 
    'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']]

    df_swap.loc[df['WLoc'] == 'H', 'WLoc'] = 'A'
    df_swap.loc[df['WLoc'] == 'A', 'WLoc'] = 'H'
    df.columns.values[6] = 'location'
    df_swap.columns.values[6] = 'location'    
      
    df.columns = [column.replace('W','T1_').replace('L','T2_') for column in df.columns]
    df_swap.columns = [column.replace('L','T1_').replace('W','T2_') for column in df_swap.columns]

    df_concat = pd.concat([df, df_swap]).reset_index(drop=True)
    
    # -1, 0, 1 encoding
    df_concat.loc[df_concat.location=='N','location'] = '0'
    df_concat.loc[df_concat.location=='H','location'] = '1'
    df_concat.loc[df_concat.location=='A','location'] = '-1'
    df_concat.location = df_concat.location.astype(int)
    
    df_concat['PointDiff'] = df_concat['T1_Score'] - df_concat['T2_Score']
    
    return df_concat

In [ ]:
# Reload data ot put into the function
regular_data = prepare_data(regular_detail_reload)
tourney_data = prepare_data(tourney_detail_reload)

In [ ]:
regular_data.shape

In [ ]:
tourney_data.shape

In [ ]:
regular_data.head()

In [ ]:
tourney_data.head()

# Feature engineering

In [ ]:
tourney_data.columns

In [ ]:
# 성능에 안좋은 영향을 미치는 속성
# FTM - free throws made (by the winning team) - bad
# FTA - free throws attempted (by the winning team) - good
# DR - defensive rebounds (pulled by the winning team)

boxscore_cols = [
        'T1_FGM', 'T1_FGA', 'T1_FGM3', 'T1_FGA3', 'T1_FTM', 'T1_FTA', 'T1_OR', 'T1_DR', 'T1_Ast', 'T1_TO', 'T1_Stl', 'T1_Blk', 'T1_PF', 
        'T2_FGM', 'T2_FGA', 'T2_FGM3', 'T2_FGA3', 'T2_FTM', 'T2_FTA', 'T2_OR', 'T2_DR', 'T2_Ast', 'T2_TO', 'T2_Stl', 'T2_Blk', 'T2_PF', 
        'PointDiff']

'''boxscore_cols = [
        'T1_FGM', 'T1_FGA', 'T1_FGM3', 'T1_FGA3', 'T1_OR', 'T1_Ast', 'T1_TO', 'T1_Stl', 'T1_PF', 
        'T2_FGM', 'T2_FGA', 'T2_FGM3', 'T2_FGA3', 'T2_OR', 'T2_Ast', 'T2_TO', 'T2_Stl', 'T2_Blk',  
        'PointDiff']'''

funcs = [np.mean]

In [ ]:
season_team_boxscore_statistics = regular_data.groupby(["Season", 'T1_TeamID'])[boxscore_cols].agg(funcs).reset_index()
season_team_boxscore_statistics.head()

In [ ]:
season_team_boxscore_statistics.columns = [''.join(col).strip() for col in season_team_boxscore_statistics.columns.values]
season_team_boxscore_statistics.head()

In [ ]:
season_T1_boxscore_stat = season_team_boxscore_statistics.copy()
season_T2_boxscore_stat = season_team_boxscore_statistics.copy()

season_T1_boxscore_stat.columns = ["T1_" + column.replace("T1_","").replace("T2_","opponent_") for column in season_T1_boxscore_stat.columns]
season_T2_boxscore_stat.columns = ["T2_" + column.replace("T1_","").replace("T2_","opponent_") for column in season_T2_boxscore_stat.columns]
season_T1_boxscore_stat.columns.values[0] = "Season"
season_T2_boxscore_stat.columns.values[0] = "Season"

In [ ]:
season_T1_boxscore_stat.head()

In [ ]:
season_T2_boxscore_stat.head()

In [ ]:
tourney_data.head()

In [ ]:
tourney_data = tourney_data[['Season', 'DayNum', 'T1_TeamID', 'T1_Score', 'T2_TeamID' ,'T2_Score']]
tourney_data.head()

In [ ]:
tourney_data = pd.merge(tourney_data, season_T1_boxscore_stat, on = ['Season', 'T1_TeamID'], how = 'left')
tourney_data = pd.merge(tourney_data, season_T2_boxscore_stat, on = ['Season', 'T2_TeamID'], how = 'left')

In [ ]:
tourney_data.head()

In [ ]:
# Extract the last 14days statistics (132 - 14 = 118)
last14days_stats_T1 = regular_data.loc[regular_data.DayNum>118].reset_index(drop=True)
last14days_stats_T1['win'] = np.where(last14days_stats_T1['PointDiff']>0,1,0)
# Winning rate of each team according to the season
last14days_stats_T1 = last14days_stats_T1.groupby(['Season','T1_TeamID'])['win'].mean().reset_index(name='T1_win_ratio_14d')

last14days_stats_T2 = regular_data.loc[regular_data.DayNum>118].reset_index(drop=True)
last14days_stats_T2['win'] = np.where(last14days_stats_T2['PointDiff']<0,1,0)
last14days_stats_T2 = last14days_stats_T2.groupby(['Season','T2_TeamID'])['win'].mean().reset_index(name='T2_win_ratio_14d')

In [ ]:
last14days_stats_T1

In [ ]:
last14days_stats_T2

In [ ]:
tourney_data.head(3)

In [ ]:
tourney_data = pd.merge(tourney_data, last14days_stats_T1, on = ['Season', 'T1_TeamID'], how = 'left')
tourney_data = pd.merge(tourney_data, last14days_stats_T2, on = ['Season', 'T2_TeamID'], how = 'left')

In [ ]:
tourney_data

In [ ]:
regular_data.head(3)

In [ ]:
regular_season_effects = regular_data[['Season','T1_TeamID','T2_TeamID','PointDiff']].copy()
regular_season_effects['T1_TeamID'] = regular_season_effects['T1_TeamID'].astype(str)
regular_season_effects['T2_TeamID'] = regular_season_effects['T2_TeamID'].astype(str)
regular_season_effects['win'] = np.where(regular_season_effects['PointDiff']>0,1,0)

In [ ]:
regular_season_effects

In [ ]:
march_madness = pd.merge(seeds[['Season','TeamID']],seeds[['Season','TeamID']],on='Season')
march_madness

In [ ]:
march_madness.columns = ['Season', 'T1_TeamID', 'T2_TeamID']
march_madness.T1_TeamID = march_madness.T1_TeamID.astype(str)
march_madness.T2_TeamID = march_madness.T2_TeamID.astype(str)

# Extract tourney result from all result (including regular season)
# Default how='inner'
regular_season_effects = pd.merge(regular_season_effects, march_madness, on = ['Season','T1_TeamID','T2_TeamID'])
regular_season_effects

In [ ]:
formula = 'win~-1+T1_TeamID+T2_TeamID'
glm = sm.GLM.from_formula(formula=formula, 
                          data=regular_season_effects.loc[regular_season_effects.Season==2014,:], 
                          family=sm.families.Binomial()).fit()

glm

In [ ]:
glm.params

In [ ]:
tourney_data

In [ ]:
seeds.head()

In [ ]:
seeds['Seed'] = seeds['Seed'].apply(lambda x: int(x[1:3]))
seeds.head()

In [ ]:
seeds_T1 = seeds[['Season','TeamID','Seed']].copy()
seeds_T2 = seeds[['Season','TeamID','Seed']].copy()
seeds_T1.columns = ['Season','T1_TeamID','T1_seed']
seeds_T2.columns = ['Season','T2_TeamID','T2_seed']

In [ ]:
tourney_data = pd.merge(tourney_data, seeds_T1, on = ['Season', 'T1_TeamID'], how = 'left')
tourney_data = pd.merge(tourney_data, seeds_T2, on = ['Season', 'T2_TeamID'], how = 'left')

In [ ]:
tourney_data["Seed_diff"] = tourney_data["T1_seed"] - tourney_data["T2_seed"]
tourney_data

# Modeling

In [ ]:
y = tourney_data['T1_Score'] - tourney_data['T2_Score']
y.describe()

In [ ]:
season_T1_boxscore_stat

In [ ]:
season_T1_boxscore_stat.columns

In [ ]:
seeds_T1.columns

In [ ]:
tourney_data

In [ ]:
# Add Score Sum Feature
season_team_score_sum = regular_data.groupby(['Season', 'T1_TeamID'])['T1_Score'].sum().reset_index()
season_team_score_sum.rename(columns={'T1_TeamID': 'TeamID', 'T1_Score': 'Score_Sum'}, inplace=True)
season_team_score_sum

In [ ]:
tourney_data = pd.merge(tourney_data, season_team_score_sum,
                        how='left',
                        left_on=['Season', 'T1_TeamID'],
                        right_on=['Season', 'TeamID'])
tourney_data.rename(columns={'Score_Sum': 'T1_Score_Sum'}, inplace=True)
tourney_data.drop(['TeamID'], axis=1, inplace=True)
tourney_data

In [ ]:
tourney_data = pd.merge(tourney_data, season_team_score_sum,
                        how='left',
                        left_on=['Season', 'T2_TeamID'],
                        right_on=['Season', 'TeamID'])
tourney_data.rename(columns={'Score_Sum': 'T2_Score_Sum'}, inplace=True)
tourney_data.drop(['TeamID'], axis=1, inplace=True)
tourney_data

In [ ]:
features = ['T1_seed', 'T2_seed', 'Seed_diff', 'T1_Score_Sum', 'T2_Score_Sum', 'T1_PointDiffmean',
       'T2_PointDiffmean', 'T1_opponent_FGAmean', 'T1_opponent_FTMmean',
       'T1_ORmean', 'T2_opponent_FGAmean', 'T1_Blkmean', 'T1_Stlmean',
       'T1_PFmean', 'T2_Blkmean', 'T2_FGMmean', 'T1_TOmean', 'T1_FGMmean',
       'T1_Astmean', 'T1_opponent_DRmean', 'T2_opponent_DRmean',
       'T2_opponent_FTMmean', 'T1_opponent_PFmean', 'T2_FGAmean',
       'T1_FGAmean', 'T2_opponent_ORmean', 'T2_Astmean',
       'T2_opponent_PFmean', 'T2_ORmean', 'T2_PFmean',
       'T1_opponent_FTAmean', 'T2_FGM3mean', 'T2_DRmean',
       'T1_opponent_ORmean', 'T2_FGA3mean', 'T2_opponent_Stlmean',
       'T2_win_ratio_14d', 'T1_opponent_Stlmean', 'T1_win_ratio_14d',
       'T1_FGM3mean', 'T2_TOmean', 'T2_Stlmean']

X = tourney_data[features].values
dtrain = xgb.DMatrix(X, label = y)

In [ ]:
def cauchyobj(preds, dtrain):
    labels = dtrain.get_label()
    c = 5000 
    x =  preds-labels    
    grad = x / (x**2/c**2+1)
    hess = -c**2*(x**2-c**2)/(x**2+c**2)**2
    return grad, hess

In [ ]:
param = {} 
#param['objective'] = 'reg:linear'
param['eval_metric'] =  'mae'
param['booster'] = 'gbtree'
param['eta'] = 0.05 #change to ~0.02 for final run
param['subsample'] = 0.35
param['colsample_bytree'] = 0.7
param['num_parallel_tree'] = 10 #recommend 10
param['min_child_weight'] = 40
param['gamma'] = 10
param['max_depth'] =  3
param['silent'] = 1

print(param)

In [ ]:
xgb_cv = []
repeat_cv = 10 # recommend 10

for i in range(repeat_cv): 
    print(f"Fold repeater {i}")
    xgb_cv.append(
        xgb.cv(
          params = param,
          dtrain = dtrain,
          obj = cauchyobj,
          num_boost_round = 3000,
          folds = KFold(n_splits = 5, shuffle = True, random_state = i),
          early_stopping_rounds = 25,
          verbose_eval = 50
        )
    )

In [ ]:
xgb_cv

In [ ]:
iteration_counts = [np.argmin(x['test-mae-mean'].values) for x in xgb_cv]
val_mae = [np.min(x['test-mae-mean'].values) for x in xgb_cv]
iteration_counts, val_mae

In [ ]:
# SHAP
import shap
# SHAP

oof_preds = []

# SHAP
total_shap_df  = pd.DataFrame()
# SHAP

for i in range(repeat_cv):
    print(f"Fold repeater {i}")
    preds = y.copy()
    kfold = KFold(n_splits = 5, shuffle = True, random_state = i)    
    
    # SHAP
    feat_importance_df  = pd.DataFrame()
    # SHAP
    
    for i, (train_index, val_index) in enumerate(kfold.split(X,y)):
        dtrain_i = xgb.DMatrix(X[train_index], label = y[train_index])
        dval_i = xgb.DMatrix(X[val_index], label = y[val_index])  
        model = xgb.train(
              params = param,
              dtrain = dtrain_i,
              num_boost_round = iteration_counts[i],
              verbose_eval = 50
        )
        
        # SHAP
        shap_values = shap.TreeExplainer(model).shap_values(X[train_index])

        fold_importance_df  = pd.DataFrame()
        fold_importance_df['feature'] = tourney_data[features].columns
        fold_importance_df['shap_values'] = abs(np.array(shap_values)[:, :].mean(0))
        # fold_importance_df['shap_values'] = abs(np.array(shap_values)[:, :].mean(1).mean(0))
        # fold_importance_df['feat_imp'] = model.feature_importance
        if i == 0:
            feat_importance_df = fold_importance_df
        else:
            feat_importance_df = pd.concat([feat_importance_df, fold_importance_df])
        # SHAP

        preds[val_index] = model.predict(dval_i)
    oof_preds.append(np.clip(preds,-30,30))
    
    # SHAP
    feat_importance_df_shap = feat_importance_df.groupby('feature').mean().sort_values('shap_values', ascending=False).reset_index()
    total_shap_df = pd.concat([total_shap_df, feat_importance_df_shap])
    # SHAP

In [ ]:
shap_sorted_df = total_shap_df.groupby('feature').mean().sort_values('shap_values', ascending=False).reset_index()
shap_sorted_df

In [ ]:
shap_sorted_df[:40].feature.values

In [ ]:
'''new_train = train[top_features].copy()
new_test = test[top_features].copy()'''

In [ ]:
y

In [ ]:
oof_preds

In [ ]:
oof_preds[0]

In [ ]:
plot_df = pd.DataFrame({"pred":oof_preds[0], "label":np.where(y>0,1,0)})
plot_df["pred_int"] = plot_df["pred"].astype(int)
plot_df = plot_df.groupby('pred_int')['label'].mean().reset_index(name='average_win_pct')

plt.figure()
plt.plot(plot_df.pred_int,plot_df.average_win_pct)

In [ ]:
spline_model = []

for i in range(repeat_cv):
    dat = list(zip(oof_preds[i],np.where(y>0,1,0)))
    dat = sorted(dat, key = lambda x: x[0])
    datdict = {}
    for k in range(len(dat)):
        datdict[dat[k][0]]= dat[k][1]
        
    spline_model.append(UnivariateSpline(list(datdict.keys()), list(datdict.values())))
    spline_fit = spline_model[i](oof_preds[i])
    
    print(f"logloss of cvsplit {i}: {log_loss(np.where(y>0,1,0),spline_fit)}") 

In [ ]:
plot_df = pd.DataFrame({"pred":oof_preds[0], "label":np.where(y>0,1,0), "spline":spline_model[0](oof_preds[0])})
plot_df["pred_int"] = (plot_df["pred"]).astype(int)
plot_df = plot_df.groupby('pred_int')['spline','label'].mean().reset_index()

plt.figure()
plt.plot(plot_df.pred_int,plot_df.spline)
plt.plot(plot_df.pred_int,plot_df.label)

In [ ]:
plot_df

In [ ]:
spline_model = []

for i in range(repeat_cv):
    dat = list(zip(oof_preds[i],np.where(y>0,1,0)))
    dat = sorted(dat, key = lambda x: x[0])
    datdict = {}
    for k in range(len(dat)):
        datdict[dat[k][0]]= dat[k][1]
    spline_model.append(UnivariateSpline(list(datdict.keys()), list(datdict.values())))
    spline_fit = spline_model[i](oof_preds[i])
    spline_fit = np.clip(spline_fit,0.025,0.975)
    
    print(f"adjusted logloss of cvsplit {i}: {log_loss(np.where(y>0,1,0),spline_fit)}") 

In [ ]:
spline_model = []

for i in range(repeat_cv):
    dat = list(zip(oof_preds[i],np.where(y>0,1,0)))
    dat = sorted(dat, key = lambda x: x[0])
    datdict = {}
    for k in range(len(dat)):
        datdict[dat[k][0]]= dat[k][1]
    spline_model.append(UnivariateSpline(list(datdict.keys()), list(datdict.values())))
    spline_fit = spline_model[i](oof_preds[i])
    spline_fit = np.clip(spline_fit,0.025,0.975)
    spline_fit[(tourney_data.T1_seed==1) & (tourney_data.T2_seed==16)] = 1.0
    spline_fit[(tourney_data.T1_seed==16) & (tourney_data.T2_seed==1)] = 0.0
    
    print(f"adjusted logloss of cvsplit {i}: {log_loss(np.where(y>0,1,0),spline_fit)}") 

In [ ]:
spline_model

In [ ]:
spline_fit

In [ ]:
#looking for upsets
pd.concat(
    [tourney_data[(tourney_data.T1_seed==1) & (tourney_data.T2_seed==16) & (tourney_data.T1_Score < tourney_data.T2_Score)],
     tourney_data[(tourney_data.T1_seed==16) & (tourney_data.T2_seed==1) & (tourney_data.T1_Score > tourney_data.T2_Score)]]
).sort_values(['Season'])

#https://en.wikipedia.org/wiki/NCAA_Division_I_Women%27s_Basketball_Tournament_upsets

In [ ]:
spline_model = []

for i in range(repeat_cv):
    dat = list(zip(oof_preds[i],np.where(y>0,1,0)))
    dat = sorted(dat, key = lambda x: x[0])
    datdict = {}
    for k in range(len(dat)):
        datdict[dat[k][0]]= dat[k][1]
    spline_model.append(UnivariateSpline(list(datdict.keys()), list(datdict.values())))
    spline_fit = spline_model[i](oof_preds[i])
    spline_fit = np.clip(spline_fit,0.025,0.975)
    spline_fit[(tourney_data.T1_seed==1) & (tourney_data.T2_seed==16) & (tourney_data.T1_Score > tourney_data.T2_Score)] = 1.0
    spline_fit[(tourney_data.T1_seed==16) & (tourney_data.T2_seed==1) & (tourney_data.T1_Score < tourney_data.T2_Score)] = 0.0
    
    print(f"adjusted logloss of cvsplit {i}: {log_loss(np.where(y>0,1,0),spline_fit)}") 

In [ ]:
val_cv = []
spline_model = []

for i in range(repeat_cv):
    dat = list(zip(oof_preds[i],np.where(y>0,1,0)))
    dat = sorted(dat, key = lambda x: x[0])
    datdict = {}
    for k in range(len(dat)):
        datdict[dat[k][0]]= dat[k][1]
    spline_model.append(UnivariateSpline(list(datdict.keys()), list(datdict.values())))
    spline_fit = spline_model[i](oof_preds[i])
    spline_fit = np.clip(spline_fit,0.025,0.975)
    spline_fit[(tourney_data.T1_seed==1) & (tourney_data.T2_seed==16) & (tourney_data.T1_Score > tourney_data.T2_Score)] = 1.0
    spline_fit[(tourney_data.T1_seed==2) & (tourney_data.T2_seed==15) & (tourney_data.T1_Score > tourney_data.T2_Score)] = 1.0
    spline_fit[(tourney_data.T1_seed==16) & (tourney_data.T2_seed==1) & (tourney_data.T1_Score < tourney_data.T2_Score)] = 0.0
    spline_fit[(tourney_data.T1_seed==15) & (tourney_data.T2_seed==2) & (tourney_data.T1_Score < tourney_data.T2_Score)] = 0.0
    
    val_cv.append(pd.DataFrame({"y":np.where(y>0,1,0), "pred":spline_fit, "season":tourney_data.Season}))
    print(f"adjusted logloss of cvsplit {i}: {log_loss(np.where(y>0,1,0),spline_fit)}") 
    
val_cv = pd.concat(val_cv)
val_cv.groupby('season').apply(lambda x: log_loss(x.y, x.pred))

# Submission time!

In [ ]:
submission

In [ ]:
submission["Season"] = submission["ID"].apply(lambda x: x[:4]).astype(int)
submission["T1_TeamID"] = submission["ID"].apply(lambda x: x[5:9]).astype(int)
submission["T2_TeamID"] = submission["ID"].apply(lambda x: x[10:14]).astype(int)
submission

In [ ]:
submission = pd.merge(submission, season_T1_boxscore_stat, on = ['Season', 'T1_TeamID'])
submission = pd.merge(submission, season_T2_boxscore_stat, on = ['Season', 'T2_TeamID'])
submission = pd.merge(submission, seeds_T1, on = ['Season', 'T1_TeamID'])
submission = pd.merge(submission, seeds_T2, on = ['Season', 'T2_TeamID'])
submission = pd.merge(submission, last14days_stats_T1, on = ['Season', 'T1_TeamID'])
submission = pd.merge(submission, last14days_stats_T2, on = ['Season', 'T2_TeamID'])
submission["Seed_diff"] = submission["T1_seed"] - submission["T2_seed"]

In [ ]:
# Add Score Sum Feature
season_team_score_sum = regular_data.groupby(['Season', 'T1_TeamID'])['T1_Score'].sum().reset_index()
season_team_score_sum.rename(columns={'T1_TeamID': 'TeamID', 'T1_Score': 'Score_Sum'}, inplace=True)
season_team_score_sum

submission = pd.merge(submission, season_team_score_sum,
                        how='left',
                        left_on=['Season', 'T1_TeamID'],
                        right_on=['Season', 'TeamID'])
submission.rename(columns={'Score_Sum': 'T1_Score_Sum'}, inplace=True)
submission.drop(['TeamID'], axis=1, inplace=True)

submission = pd.merge(submission, season_team_score_sum,
                        how='left',
                        left_on=['Season', 'T2_TeamID'],
                        right_on=['Season', 'TeamID'])
submission.rename(columns={'Score_Sum': 'T2_Score_Sum'}, inplace=True)
submission.drop(['TeamID'], axis=1, inplace=True)

In [ ]:
submission

In [ ]:
X_submission = submission[features].values
dtest = xgb.DMatrix(X_submission)

In [ ]:
sub_models = []
for i in range(repeat_cv):
    print(f"Fold repeater {i}")
    sub_models.append(
        xgb.train(
          params = param,
          dtrain = dtrain,
          num_boost_round = int(iteration_counts[i] * 1.05),
          verbose_eval = 50
        )
    )

In [ ]:
sub_preds = []
for i in range(repeat_cv):
    sub_preds.append(np.clip(spline_model[i](np.clip(sub_models[i].predict(dtest),-30,30)),0.025,0.975))
    
submission["Pred"] = pd.DataFrame(sub_preds).mean(axis=0)

submission.loc[(submission.T1_seed==1) & (submission.T2_seed==16), 'Pred'] = 1.0
submission.loc[(submission.T1_seed==16) & (submission.T2_seed==1), 'Pred'] = 0.0

if COMPETITION_YEAR == 2019:
    submission[['ID','Pred']].to_csv("submission-Men-2019.csv", index = None)
elif COMPETITION_YEAR == 2020:
    submission[['ID','Pred']].to_csv("submission-Men-2020.csv", index = None)

In [ ]:
submission['Pred'].hist()
plt.title('Prediction Distribution');